In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from math import *
torch.cuda.set_device(0)

In [2]:
class ResNet(nn.Module):
    def __init__(self,dim, m,o):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        self.Ix[0,0] = 1
        self.Ix[1,1] = 1
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        
        self.outlayer = nn.Linear(m, o)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y**deg)
        y = self.fc2(y)
        y = F.relu(y**deg)
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y**deg)
        y = self.fc4(y)
        y = F.relu(y**deg)
        y = y+s
        
        
        output = self.outlayer(y)
        return output
deg = 2

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
def sol_exact(x):
    N_norm_s = torch.sum(x**2,dim=1) 
    N_u = (torch.exp(N_norm_s)).reshape([x.size()[0],1])
    return N_u
def sol_exact_H1(x):
    N_norm_s = torch.sum(x**2,dim=1)
    u = sol_exact(x)
    v= torch.ones(u.shape).cuda()
    sol = torch.zeros(x.size()[0],dim_set+1).cuda()
    sol[:,0] = u.reshape([1,x.size()[0]])
    sol[:,1:dim_set+1] = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    return sol

In [5]:
def right(x):
    N_norm_s = torch.sum(x**2,dim=1).reshape([x.size()[0],1])
    N_u = torch.exp(N_norm_s)
    f_tem = 2*dim_set * N_u + 4 * N_norm_s *N_u + N_u**2
    return f_tem 

In [6]:
Batch_size = 10000
dim_set = 8
def DRM():
    x = generate_uniform_points_in_sphere(dim_set,1,Batch_size).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    ux = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    loss = torch.sum((ux-du)**2)/Batch_size
    uxx = torch.zeros(Batch_size,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,i] = uxx_tem[:,i]
    loss = loss + torch.sum((torch.sum(uxx,dim=1).reshape([x.size()[0],1]) +u**2 - right(x))**2)/Batch_size
    return loss

In [7]:
def generate_uniform_points_in_sphere(d,R,N):
    points = torch.randn(N,d)
    scales = (R*torch.rand(N))**(1/d)
    notes = points/(torch.sqrt(torch.sum(points**2,dim=1))/scales).reshape(N,1)*torch.ones(N ,d)
    return notes

In [8]:
model_u1 = ResNet(dim_set,20,1)
model_p = ResNet(dim_set,20,dim_set)

device=torch.device("cuda:0" )
model_u1.to(device)
model_p.to(device)

ResNet(
  (fc1): Linear(in_features=8, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=20, bias=True)
  (outlayer): Linear(in_features=20, out_features=8, bias=True)
)

In [9]:
def model_u(x):
    return (torch.sum(x**2,dim=1)-1).reshape([x.size()[0],1])*model_u1(x) + exp(1)

In [10]:
def loss_error():
    notes_error = generate_uniform_points_in_sphere(dim_set,1,10000).cuda()
    notes_error.requires_grad= True
    exact = sol_exact_H1(notes_error)
    u_L2 = sqrt(torch.sum((exact[:,0])**2)/10000)
    Du_L2 = sqrt(torch.sum(exact[:,1:dim_set+1]**2)/10000)
    predict = torch.zeros(10000,2*dim_set+2).cuda()
    predict[:,0]= model_u(notes_error).reshape([1,notes_error.size()[0]])
    predict[:,1:dim_set+1]= model_p(notes_error)
    value1 = sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    value2 = sqrt(torch.sum((predict[:,1:dim_set+1] - exact[:,1:dim_set+1])**2)/10000)/Du_L2
    return value1,value2

In [11]:
traintime = 20000
error_save=np.zeros([2,traintime])
optimizer = optim.Adam([
                {'params': model_u1.parameters()},
                {'params': model_p.parameters()}
            ])
i = 0
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses = DRM()
    losses.backward()
    optimizer.step() 
    error1,error2 = loss_error()
    error_save[0,i]=float(error1)
    error_save[1,i]=float(error2)
    i = i +1
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error1)
        print("loss1 =",losses.detach())
        np.save("error_save_LDG_La_8D_w20.npy", error_save)
np.save("error_save_LDG_La_8D_w20.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = 0.2566494283789292
loss1 = tensor(1774.0852, device='cuda:0')
i=  501
error = 0.0185030177753644
loss1 = tensor(11.9718, device='cuda:0')
i=  1001
error = 0.010459347015671468
loss1 = tensor(4.2992, device='cuda:0')
i=  1501
error = 0.008837929468572987
loss1 = tensor(1.5478, device='cuda:0')
i=  2001
error = 0.007223420260506018
loss1 = tensor(0.7515, device='cuda:0')
i=  2501
error = 0.005452726284406032
loss1 = tensor(0.4414, device='cuda:0')
i=  3001
error = 0.0041258559585663195
loss1 = tensor(0.2802, device='cuda:0')
i=  3501
error = 0.0033282054985001703
loss1 = tensor(0.1967, device='cuda:0')
i=  4001
error = 0.002984375117639101
loss1 = tensor(0.2004, device='cuda:0')
i=  4501
error = 0.0024197436692125996
loss1 = tensor(0.1150, device='cuda:0')
i=  5001
error = 0.0022140589485136297
loss1 = tensor(0.0886, device='cuda:0')
i=  5501
error = 0.001934955469781513
loss1 = tensor(0.0734, device='cuda:0')
i=  6001
error = 0.0018261761938472563
loss1 = tensor(0.0687, de

In [12]:
import matplotlib.pyplot as plt